## Parameters

In [335]:
# Where the data is located and the engineered features will be saved.
data_path = '../00_data/'

## Import dependencies

In [336]:
# display full output in Notebook, instead of only the last result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# standard libraries
import numpy as np
import pandas as pd
import os
from datetime import datetime

# make this notebook's output stable across runs
np.random.seed(42)
    
# ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## Import data

In [337]:
# file names
rapperswil_data = 'rapperswil.csv'
burgdorf_data = 'burgdorf.csv'
weather_rapperswil = 'weather_rapperswil.csv'
weather_burgdorf = 'weather_burgdorf.csv'
public_holidays_rapperswil = 'public_holidays_zurich.csv'
public_holidays_bern = 'public_holidays_bern.csv'
ticket_sales_rapperswil = 'ticket_sales_rapperswil.csv'
ticket_sales_burgdorf = 'ticket_sales_burgdorf.csv'

# function to import data
def load_data(data_path, data_file, **kwargs):
    csv_path = os.path.join(data_path, data_file)
    return pd.read_csv(csv_path, **kwargs)

# load weather data
df_weather_rapperswil = load_data(data_path, weather_rapperswil, sep=',')
df_weather_burgdorf = load_data(data_path, weather_burgdorf, sep=',')

# load parking data
df_rapperswil = load_data(data_path, rapperswil_data, sep=',')
df_burgdorf = load_data(data_path, burgdorf_data, sep=';', usecols=['date', 'occupancy_rate'])

# load ticket sales data
df_sales_rapperswil = load_data(data_path, ticket_sales_rapperswil, sep = ';', usecols=['Start'])
df_sales_burgdorf = load_data(data_path, ticket_sales_burgdorf, sep = ';', usecols=['Start'])

# load public holiday data
# columns to keep
df_holidays_rapperswil = load_data(data_path, public_holidays_rapperswil, sep=';', encoding='latin1', usecols=['Date'])
df_holidays_burgdorf = load_data(data_path, public_holidays_bern, sep=';', encoding='latin1', usecols=['Date'])

print('Dataset length of Rapperswil data: {}'.format(len(df_rapperswil)))
print('Dataset length of Burgdorf data: {}'.format(len(df_burgdorf)))

Dataset length of Rapperswil data: 12324
Dataset length of Burgdorf data: 7789


## Data cleaning 

#### 0) Public holiday data

source: https://www.feiertagskalender.ch/

columns:
- **date:** represents a public holiday 

In [338]:
# Change Date column to date and convert to timestamp
df_holidays_rapperswil = df_holidays_rapperswil.rename(columns={'Date' : 'date'})
df_holidays_rapperswil['date'] = [datetime.strptime(i, '%d.%m.%Y').date() for i in df_holidays_rapperswil.date]

df_holidays_burgdorf = df_holidays_burgdorf.rename(columns={'Date' : 'date'})
df_holidays_burgdorf['date'] = [datetime.strptime(i, '%d.%m.%Y').date() for i in df_holidays_burgdorf.date]

#### 1) Parking data

columns:
- **datetime:** hourly
- **occupancy_rate:** avg. parking occupancy for given hour

In [339]:
#rename columns
df_rapperswil = df_rapperswil.rename(columns={'DATE': 'date', 'BELEGUNGSQUOTE (%)': 'occupancy_rate'})
df_burgdorf = df_burgdorf.rename(columns={'category': 'date', 'Auslastung': 'occupancy_rate'})

#convert date column into datetime format
df_rapperswil['date'] = pd.to_datetime(df_rapperswil['date'])
df_burgdorf['date'] = pd.to_datetime(df_burgdorf['date'])

#remove time zone
df_rapperswil['date'] = df_rapperswil['date'].apply(lambda x: x.replace(tzinfo=None))

**2) Ticket sales data**

In [340]:
def ticket_sales_cleaning(df_sales):
    
    # delete time zone information
    df_sales['Start'] = pd.to_datetime(df_sales['Start'])
    df_sales['Start'] = df_sales['Start'].apply(lambda x: x.replace(tzinfo=None))

    # change column name
    df_sales = df_sales.rename(columns={'Start': 'date'})

    # round down to the hour
    df_sales['date'] = df_sales['date'].apply(lambda x: x.replace(microsecond=0, second=0, minute=0))

    # use index as column for a more convenient aggregation
    df_sales['id'] = df_sales.index

    # create new df by aggregating ticket sales per full hour
    df_sales = pd.DataFrame(df_sales.groupby(['date'])['id'].count())

    # rename column
    df_sales = df_sales.rename(columns={'id': 'sales'})
    
    # set index as new column
    df_sales['date'] = df_sales.index
    
    df_sales.reset_index(drop=True, inplace=True)

    return df_sales


In [341]:
# apply cleaning function to both ticket sales data set
df_sales_rapperswil = ticket_sales_cleaning(df_sales_rapperswil)
df_sales_burgdorf = ticket_sales_cleaning(df_sales_burgdorf)

#### 2) Weather data

source: https://home.openweathermap.org/history_bulks/new (fee required)

columns:
- **temperature** (kelvin)
- **weather:** rain, clear, clouds, etc.
- **datetime:** hourly

In [342]:
# only keep essential columns
df_weather_rapperswil = df_weather_rapperswil[['dt', 'temp', 'weather_main']]
df_weather_burgdorf = df_weather_burgdorf[['dt', 'temp', 'weather_main']]

# convert date column into datetime format
df_weather_rapperswil['date'] = pd.to_datetime(df_weather_rapperswil['dt'], unit='s')
df_weather_burgdorf['date'] = pd.to_datetime(df_weather_burgdorf['dt'], unit='s')

# drop dt column
df_weather_rapperswil.drop(columns=['dt'], inplace = True)
df_weather_burgdorf.drop(columns=['dt'], inplace = True)

# rename columns
df_weather_rapperswil = df_weather_rapperswil.rename(columns={'weather_main': 'weather', 'temp' : 'temperature'})
df_weather_burgdorf = df_weather_burgdorf.rename(columns={'weather_main': 'weather', 'temp' : 'temperature'})

## Feature Engineering

We see the following variables as relevant for feature-based models:

**Features based on internal data:**
- Hour
- Day of week
- Quarter
- Month
- Day of year
- Day of month
- Week of year
- occupancy rate $t-1$
- occupancy rate $t-2$
- occupancy rate $t-3$
- occupancy rate $t-7$
- ticket sales $t-1$
- ticket sales $t-2$
- ticket sales $t-3$
- ticket sales $t-7$

**Features based on external data:**
- Weather type (clouds, rain, snow, etc.)
- Weather type $t-1$
- Weather type $t-2$
- Weather type $t-3$
- Weather type $t-7$
- Temperature (Kelvin)
- Temperature (Kelvin) $t-1$
- Temperature (Kelvin) $t-2$
- Temperature (Kelvin) $t-3$
- Temperature (Kelvin) $t-7$
- Public holiday (boolean value)




### Features based on internal data

In [343]:
def time_features(df):
    """
    Input parking data.
    Creates several time dimensions, such as quarter or day of week.
    Return df with historical features.
    """
    df = df.copy()
    df['date_only'] = df['date'].dt.date # only for visualization purposes, not a feature
    df['hour'] = df['date'].dt.hour
    df['day_of_week'] = df['date'].dt.dayofweek
    df['day_name'] = df['date'].dt.day_name()
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['day_of_year'] = df['date'].dt.dayofyear
    df['day_of_month'] = df['date'].dt.day
    df['week_of_year'] = df['date'].dt.weekofyear
    
    return df

In [344]:
# create historical features
df_rapperswil = time_features(df_rapperswil)
df_burgdorf = time_features(df_burgdorf)

Merge parking data with ticket sales data and add lag features

In [345]:
# merge ticket sales data with parking data
df_rapperswil = pd.merge(df_rapperswil, df_sales_rapperswil, how='left', on='date')
df_burgdorf = pd.merge(df_burgdorf, df_sales_burgdorf, how='left', on='date')

# replace nan values in sales column with 0
df_rapperswil['sales'] = df_rapperswil['sales'].fillna(0)
df_burgdorf['sales'] = df_burgdorf['sales'].fillna(0)

In [347]:
len(df_rapperswil)

12324

In [278]:
def lag_features_sales(df_p):
    df_park = df_p.copy()

    # prepare lag data 
    df_lags = pd.concat([df_park['sales'].shift(7),
                        df_park['sales'].shift(3),
                        df_park['sales'].shift(2),
                        df_park['sales'].shift(1)], axis=1)

    df_lags.columns = ['sales_t-7', 'sales_t-3', 'sales_t-2', 'sales_t-1']

    # join lag df with parking and lag data
    df_park = pd.concat([df_lags, df_park], axis=1)
    
    # drop rows that contain NaN values caused by shifting
    df_park.dropna(inplace=True)
    
    # drop sales column as only lag values are of interest 
    df_park.drop('sales', axis=1, inplace=True)

    # set datetime as index
    # df_park = df_park.set_index('date')

    return df_park

In [279]:
# apply lag features for sales data to both sets
df_rapperswil = lag_features_sales(df_rapperswil)
df_burgdorf = lag_features_sales(df_burgdorf)

2476
205


### Features based on external data

Data for related features (weather, holidays) already exists and, thus, needs to be merged with parking data

#### A) Merge weather data, parking and public holidays data

In [220]:
def merge_weather(df_p, df_w):
    df_park = df_p.copy()
    df_weather = df_w.copy()

    # set datetime as index for merging
    df_weather.set_index('date')
    df_park.set_index('date')

    # merge parking and weather data
    df_park = df_park.merge(df_weather, left_index=True, right_index=True)
    df_park.drop(['date_y'], axis=1, inplace=True)
    df_park = df_park.rename(columns={'date_x' : 'date'})
    df_park.set_index('date')

    return df_park

In [221]:
df_rapperswil = merge_weather(df_rapperswil, df_weather_rapperswil)
df_burgdorf = merge_weather(df_burgdorf, df_weather_burgdorf)

In [222]:
def weather_lag_features(df_p):
    df_park = df_p.copy()

    # prepare lag data for weather_type
    df_lags = pd.concat([df_park['weather'].shift(7),
                         df_park['weather'].shift(3),
                         df_park['weather'].shift(2),
                         df_park['weather'].shift(1)], axis=1)

    df_lags.columns = ['weather_t-7', 'weather_t-3', 'weather_t-2', 'weather_t-1']

    # join lag df with parking and weather data 
    df_park = pd.concat([df_lags, df_park], axis=1)
    
    
    # prepare lag data for temperature
    df_temp_lags = pd.concat([df_park['temperature'].shift(7),
                         df_park['temperature'].shift(3),
                         df_park['temperature'].shift(2),
                         df_park['temperature'].shift(1)], axis=1)

    df_temp_lags.columns = ['temperature_t-7', 'temperature_t-3', 'temperature_t-2', 'temperature_t-1']
    
    # join lag df with parking and temperature data 
    df_park = pd.concat([df_temp_lags, df_park], axis=1)

    # drop rows that contain NaN values caused by shifting
    df_park.dropna(inplace=True)

    # set datetime as index
    df_park = df_park.set_index('date')


    return df_park

In [223]:
df_rapperswil = weather_lag_features(df_rapperswil)
df_burgdorf = weather_lag_features(df_burgdorf)

#### B) Add public holiday information

In [224]:
# create boolean column by identifying whether data corresponds to a public holiday
holidays = [i for i in df_holidays_rapperswil.date]
df_rapperswil['holiday'] = df_rapperswil['date_only'].isin(holidays)

holidays = [i for i in df_holidays_burgdorf.date]
df_burgdorf['holiday'] = df_burgdorf['date_only'].isin(holidays)

# convert true/false into integers
df_rapperswil.replace({'holiday': {False: 0, True: 1}}, inplace=True)
df_burgdorf.replace({'holiday': {False: 0, True: 1}}, inplace=True)

In [225]:
print('Length and number of columns of Rapperswil data: {}, {}'.format(len(df_rapperswil), len(df_rapperswil.columns)))
print('Length and number of columns of Burgdorf data: {}, {}'.format(len(df_burgdorf), len(df_burgdorf.columns)))

Length and number of columns of Rapperswil data: 2462, 25
Length and number of columns of Burgdorf data: 191, 25


### Lag features

In [226]:
def lag_features(df_p):
    df_park = df_p.copy()

    # prepare lag data 
    df_lags = pd.concat([df_park['occupancy_rate'].shift(7),
                        df_park['occupancy_rate'].shift(3),
                        df_park['occupancy_rate'].shift(2),
                        df_park['occupancy_rate'].shift(1)], axis=1)

    df_lags.columns = ['t-7', 't-3', 't-2', 't-1']

    # join lag df with parking and lag data
    df_park = pd.concat([df_lags, df_park], axis=1)

    # drop rows that contain NaN values caused by shifting
    df_park.dropna(inplace=True)

    # set datetime as index
    # df_park = df_park.set_index('date')

    return df_park

In [227]:
df_rapperswil = lag_features(df_rapperswil)
df_burgdorf = lag_features(df_burgdorf)

In [228]:
print('Length and number of columns of Rapperswil data: {}, {}'.format(len(df_rapperswil), len(df_rapperswil.columns)))
print('Length and number of columns of Burgdorf data: {}, {}'.format(len(df_burgdorf), len(df_burgdorf.columns)))

Length and number of columns of Rapperswil data: 2455, 29
Length and number of columns of Burgdorf data: 184, 29


In [229]:
# format columns
df_rapperswil = df_rapperswil.astype({'t-7': float,
                                      't-3': float,
                                      't-2': float,
                                      't-1': float,
                                      'occupancy_rate': float})

df_rapperswil['date_only'] = pd.to_datetime(df_rapperswil['date_only'])
df_burgdorf['date_only'] = pd.to_datetime(df_burgdorf['date_only'])

In [230]:
# Print to see if everything is ok
# df_rapperswil
# df_burgdorf

## Export Feature to CSV

In [231]:
df_rapperswil.to_csv(os.path.join(data_path, "features_rapperswil.csv"), sep=",")
df_burgdorf.to_csv(os.path.join(data_path, "features_burgdorf.csv"), sep=",")